# 6He Hamiltonian Construction

This notebook constructs the Hamiltonian for 6He (4He core + 2 neutrons in p-shell) using the `ckpot.snt` interaction file.

## System Description
- **Core**: 4He
- **Valence**: 2 neutrons in p-shell ($0p_{1/2}, 0p_{3/2}$)
- **Basis States**: Pairs with $M=0$, Parity +.
  1. $|001\rangle$: $(0p_{1/2})^2, (j=1/2, m=+1/2; j=1/2, m=-1/2)$
  2. $|010\rangle$: $(0p_{3/2})^2, (j=3/2, m=+1/2; j=3/2, m=-1/2)$
  3. $|100\rangle$: $(0p_{3/2})^2, (j=3/2, m=+3/2; j=3/2, m=-3/2)$

## Method
1. Read Single Particle Energies (SPE) and Two-Body Matrix Elements (TBME) from `ckpot.snt`.
2. Transform Coupled-J TBMEs to uncoupled m-scheme pair matrix elements.
3. Construct the Hamiltonian matrix.
4. Map to Qubits/Pauli Operators using:
   $$ H = \sum_i (2\epsilon_i + V_{ii}) \frac{I_i - Z_i}{2} + \sum_{i<j} V_{ij} \frac{X_i X_j + Y_i Y_j}{2} $$

In [1]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from sympy.physics.quantum.cg import CG
import os

## 1. Parse `ckpot.snt`

In [2]:
def parse_ckpot(filepath):
    spe = {}
    tbme = {}
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    # Indices for orbits (from file header logic)
    # 1: p 0p_1/2 (proton)
    # 2: p 0p_3/2 (proton)
    # 3: n 0p_1/2 (neutron)
    # 4: n 0p_3/2 (neutron)
    # We only care about neutrons (3 and 4).
    
    reading_spe = False
    reading_tbme = False
    
    for line in lines:
        parts = line.split()
        if not parts or parts[0].startswith('!'):
            continue
            
        # Simple state machine based on expected structure
        # Valid lines have numbers.
        try:
            nums = [float(x) for x in parts]
        except ValueError:
            continue
            
        if len(nums) == 2 and int(nums[0]) == 4 and int(nums[1]) == 0:
            reading_spe = True
            continue
        if len(nums) == 3 and int(nums[0]) == 34:
            reading_spe = False
            reading_tbme = True
            continue
            
        if reading_spe:
            # format: index index energy
            if len(nums) >= 3:
                idx = int(nums[0])
                energy = nums[2]
                if idx == 3: spe['p1/2'] = energy
                if idx == 4: spe['p3/2'] = energy
                
        if reading_tbme:
            # format: i j k l J val
            if len(nums) >= 6:
                i, j, k, l = int(nums[0]), int(nums[1]), int(nums[2]), int(nums[3])
                J = int(nums[4])
                val = nums[5]
                
                # Store as tuple (i,j,k,l,J)
                # 3=p1/2, 4=p3/2
                tbme[(i,j,k,l,J)] = val
                
    return spe, tbme

spe, tbme_data = parse_ckpot('../interaction_file/ckpot.snt')
print("SPE:", spe)
# print("TBME Sample:", list(tbme_data.items())[:5])

SPE: {'p1/2': 2.419, 'p3/2': 1.129}


## 2. Calculate Hamiltonian Coefficients

### Basis Calculation (Clebsch-Gordan)
We need to convert coupled J matrix elements to the uncoupled pair basis.

Basis states:
0. $|p_{1/2}, +1/2; p_{1/2}, -1/2\rangle$
1. $|p_{3/2}, +1/2; p_{3/2}, -1/2\rangle$
2. $|p_{3/2}, +3/2; p_{3/2}, -3/2\rangle$

Expansion in Coupled J states $|j j J 0\rangle$:
$$ |j m j -m\rangle = \sum_{J} \langle j m j -m | J 0 \rangle |j^2 J 0\rangle $$

Since we deal with identical fermions, J must be even.
For $j=1/2$, Only $J=0$.
For $j=3/2$, $J=0, 2$.


In [3]:
def get_cg(j, m1, m2, J, M):
    return float(CG(j, m1, j, m2, J, M).doit())

# Coefficients for p1/2
# State 0: (1/2, 1/2, 1/2, -1/2)
c_0_J0 = get_cg(1/2, 1/2, -1/2, 0, 0)
print(f"p1/2 m=1/2 -> J=0: {c_0_J0}")

# Coefficients for p3/2
# State 1: (3/2, 1/2, 3/2, -1/2)
c_1_J0 = get_cg(3/2, 1/2, -1/2, 0, 0)
c_1_J2 = get_cg(3/2, 1/2, -1/2, 2, 0)
print(f"p3/2 m=1/2 -> J=0: {c_1_J0}, J=2: {c_1_J2}")

# State 2: (3/2, 3/2, 3/2, -3/2)
c_2_J0 = get_cg(3/2, 3/2, -3/2, 0, 0)
c_2_J2 = get_cg(3/2, 3/2, -3/2, 2, 0)
print(f"p3/2 m=3/2 -> J=0: {c_2_J0}, J=2: {c_2_J2}")

p1/2 m=1/2 -> J=0: 0.7071067811865476
p3/2 m=1/2 -> J=0: -0.5, J=2: 0.5
p3/2 m=3/2 -> J=0: 0.5, J=2: 0.5


### Matrix Elements $V_{ij}$
We calculate $V_{ab} = \langle a | V | b \rangle$ where $a, b$ are indices 0, 1, 2.
$ V_{ab} = \sum_{J} C_a(J) C_b(J) \langle j_a j_a J | V | j_b j_b J \rangle $

Note: $V$ in definition includes antisymmetrization. The file provides standard TBME.

Pairs:
- 0: $p_{1/2}$ (idx 3)
- 1: $p_{3/2}$ (idx 4)
- 2: $p_{3/2}$ (idx 4)


In [4]:
# Mapping indices to pair properties
# basis index -> (shell_idx, coeffs_dict)
basis = {
    0: {'shell': 3, 'coeffs': {0: c_0_J0}},
    1: {'shell': 4, 'coeffs': {0: c_1_J0, 2: c_1_J2}},
    2: {'shell': 4, 'coeffs': {0: c_2_J0, 2: c_2_J2}}
}

# Function to get TBME from loaded data
def get_tbme_val(i, j, k, l, J):
    # Standard SNT order usually i<=j, k<=l, ij<=kl
    # We need to try permutations if not found directly
    key = (i, j, k, l, J)
    if key in tbme_data: return tbme_data[key]
    key = (k, l, i, j, J)
    if key in tbme_data: return tbme_data[key]
    return 0.0

V_matrix = np.zeros((3, 3))

for a in range(3):
    for b in range(3):
        shell_a = basis[a]['shell']
        shell_b = basis[b]['shell']
        
        val = 0.0
        # sum over J common to both
        common_Js = set(basis[a]['coeffs'].keys()) & set(basis[b]['coeffs'].keys())
        
        for J in common_Js:
            c_a = basis[a]['coeffs'][J]
            c_b = basis[b]['coeffs'][J]
            
            # TBME for (shell_a, shell_a) -> (shell_b, shell_b) coupled to J
            mat_el = get_tbme_val(shell_a, shell_a, shell_b, shell_b, J)
            val += c_a * c_b * mat_el
            
        V_matrix[a, b] = val

print("Interaction Matrix V_ij (pair basis):")
print(V_matrix)

Interaction Matrix V_ij (pair basis):
[[ 0.122       1.78636386 -1.78636386]
 [ 1.78636386 -0.810225    0.854125  ]
 [-1.78636386  0.854125   -0.810225  ]]


### Total Hamiltonian Construction
Pair diagonal Energy $E_i = 2\epsilon_{orbit} + V_{ii}$.
Off-diagonal coupling $V_{ij}$ for $i \neq j$.

In [5]:
E_pair = np.zeros(3)

# SPE values
epsilon = {
    3: spe['p1/2'],
    4: spe['p3/2']
}

for i in range(3):
    shell = basis[i]['shell']
    E_pair[i] = 2 * epsilon[shell] + V_matrix[i, i]

print("Pair Energies (Diagonal):", E_pair)
print("Off-diagonal couplings:")
print(f"V_01: {V_matrix[0,1]}")
print(f"V_02: {V_matrix[0,2]}")
print(f"V_12: {V_matrix[1,2]}")

Pair Energies (Diagonal): [4.96     1.447775 1.447775]
Off-diagonal couplings:
V_01: 1.786363861311575
V_02: -1.786363861311575
V_12: 0.854125


## 3. Qiskit Hamiltonian

Mapping to Qubits:
- Qubit 0: State 0 ($p_{1/2}$ pair)
- Qubit 1: State 1 ($p_{3/2}, m=1/2$ pair)
- Qubit 2: State 2 ($p_{3/2}, m=3/2$ pair)
---
**NOTE**: The user requirement maps arbitrary states to superposition of $|001>, |010>, |100>$. 
This implies a **One-Hot Encoding** subspace or similar? 
Wait, "Arbitrary state is superposition of |001>, |010>, |100>".
This means we have 3 qubits, but only subspace with Hamming weight 1 is physical?
Let's re-read: "qiskit ... each qubit is pair occupancy ... |001>, |010>, |100> base".
Yes, this is ONE pair in the whole space. We are looking at 6He = 4He + 2n.
So we assume ONLY ONE pair exists in the valence space? 
Wait, 2 neutrons.
A pair is 2 neutrons.
So yes, we have 1 pair state occupied.
So the physical subspace is indeed $|001\rangle$ (qubit 0=1), $|010\rangle$ (qubit 1=1), etc.

The Hamiltonian form:
$$ H = \sum_{i} E_i \frac{I_i - Z_i}{2} + \sum_{i<j} V_{ij} \frac{X_i X_j + Y_i Y_j}{2} $$
This form naturally preserves particle number. $\frac{X_i X_j + Y_i Y_j}{2}$ is hopping $a^\dagger_i a_j + a^\dagger_j a_i$.
This will mix $|100\rangle \leftrightarrow |010\rangle$ etc.

In [6]:
ham_ops = []

# Number of qubits = 3
num_qubits = 3

# Diagonal terms: E_i * (I-Z_i)/2
for i in range(num_qubits):
    label = ['I'] * num_qubits
    label[i] = 'Z'
    z_str = "".join(reversed(label)) # Qiskit Little Endian: q2 q1 q0
    
    # Constant part E_i/2 * I
    ham_ops.append(("I"*num_qubits, E_pair[i] / 2))
    
    # Z part -E_i/2 * Z
    ham_ops.append((z_str, -E_pair[i] / 2))

# Off-diagonal terms: V_ij * (XX + YY) / 2
for i in range(num_qubits):
    for j in range(i + 1, num_qubits):
        coupling = V_matrix[i, j]
        
        # XX
        label_x = ['I'] * num_qubits
        label_x[i] = 'X'
        label_x[j] = 'X'
        x_str = "".join(reversed(label_x))
        ham_ops.append((x_str, coupling / 2))
        
        # YY
        label_y = ['I'] * num_qubits
        label_y[i] = 'Y'
        label_y[j] = 'Y'
        y_str = "".join(reversed(label_y))
        ham_ops.append((y_str, coupling / 2))

H_op = SparsePauliOp.from_list(ham_ops).simplify()
print("Hamiltonian Operator:")
print(H_op)

Hamiltonian Operator:
SparsePauliOp(['III', 'IIZ', 'IZI', 'ZII', 'IXX', 'IYY', 'XIX', 'YIY', 'XXI', 'YYI'],
              coeffs=[ 3.927775  +0.j, -2.48      +0.j, -0.7238875 +0.j, -0.7238875 +0.j,
  0.89318193+0.j,  0.89318193+0.j, -0.89318193+0.j, -0.89318193+0.j,
  0.4270625 +0.j,  0.4270625 +0.j])


In [7]:
# Verification: Print matrix representation in the relevant subspace
mat = H_op.to_matrix()
print("Full Matrix shape:", mat.shape)

# Basis states in Qiskit (Little Endian): |q2, q1, q0>
# Qubit 0 (State 0) -> |001> -> Index 1
# Qubit 1 (State 1) -> |010> -> Index 2
# Qubit 2 (State 2) -> |100> -> Index 4
indices = [1, 2, 4]
labels = ["|001> (p1/2)", "|010> (p3/2 m=1/2)", "|100> (p3/2 m=3/2)"]

print("\nSubspace Matrix:")
sub_mat = np.zeros((3, 3), dtype=complex)
for r, i in enumerate(indices):
    for c, j in enumerate(indices):
        sub_mat[r, c] = mat[i, j]
        
print(np.real(sub_mat))

print("\nTarget Pair Hamiltonian calculated earlier:")
print("Diagonal:", E_pair)
print("Off-diagonal V_ij:\n", V_matrix)

Full Matrix shape: (8, 8)

Subspace Matrix:
[[ 4.96        1.78636386 -1.78636386]
 [ 1.78636386  1.447775    0.854125  ]
 [-1.78636386  0.854125    1.447775  ]]

Target Pair Hamiltonian calculated earlier:
Diagonal: [4.96     1.447775 1.447775]
Off-diagonal V_ij:
 [[ 0.122       1.78636386 -1.78636386]
 [ 1.78636386 -0.810225    0.854125  ]
 [-1.78636386  0.854125   -0.810225  ]]


In [8]:
# Calculate Eigenvalues
eigenvalues, eigenvectors = np.linalg.eigh(sub_mat)
print("\nEigenvalues of the 6He Hamiltonian (Physical Subspace):")
for i, val in enumerate(eigenvalues):
    print(f"State {i}: E = {val:.6f} MeV")



Eigenvalues of the 6He Hamiltonian (Physical Subspace):
State 0: E = -0.562104 MeV
State 1: E = 2.301900 MeV
State 2: E = 6.115754 MeV
